# TP 3
## Punto 3
### Modelo 2

##### Importo librerias que utilizare

In [1]:
from sklearn.datasets import load_iris
import numpy as np
import math
from tqdm import tqdm
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
from sklearn.preprocessing import (
    LabelEncoder,
    OneHotEncoder,
    OrdinalEncoder,
)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, ConfusionMatrixDisplay, confusion_matrix
import xgboost as xgb
import category_encoders.utils as util
from category_encoders import TargetEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier 
from sklearn.preprocessing import StandardScaler

In [2]:
import warnings 
warnings.filterwarnings('ignore')

### Lectura de archivos

In [3]:
iden = pd.read_csv('train_identity.csv')

In [4]:
trans = pd.read_csv('train_transaction.csv')

Realizo el merge de ambos df cargados anteriormente

In [5]:
joined = trans.merge(iden,how='left')

Elimino los IDs únicos, ya que no influyen en mi predicción. Al igual que el TransactionAmt.

In [6]:
del(joined['TransactionID'])

In [7]:
del(joined['TransactionAmt'])

Libero espacio en la RAM, eliminando mis los dos df cargados inicialmente, ya que no volvere a utilizarlos.

In [8]:
del iden

In [9]:
del trans

Busco el valor para realizar el split del train y del validation, quedandome con el 80% del df para el train y con el 20% restante para el validation.

In [10]:
limite = np.percentile(joined['TransactionDT'], 80)

Filtro en el df para generar el train y el validation, y sus labels.

In [11]:
filtro_validation = joined['TransactionDT'] > limite

In [12]:
validation = joined[filtro_validation]

In [13]:
label_validation = validation[['isFraud']]

In [14]:
del(validation['isFraud'])

In [15]:
filtro_train = joined['TransactionDT']<= limite

In [16]:
train = joined[filtro_train]

In [17]:
label_train = train[['isFraud']]

In [18]:
del(train['isFraud'])

Libero espacion en la RAM, eliminando un df que ya no utilizare.

In [19]:
del joined

In [20]:
del(validation['TransactionDT'])

In [21]:
del(train['TransactionDT'])

Creo el siguiente objeto, para normalizar los features para la red neuronal.

In [22]:
scaler = StandardScaler()

Creo el diccionario donde almacenare los encodings de cada feature.

In [23]:
dict_encoding = {}

Creo las funciones que utilizare a continuacion para realizar los encodings.

In [24]:
def countVect(df,columna,dic):
    if columna not in dic:
        dic[columna] = CountVectorizer(max_features =10)
        col_encoded = dic[columna].fit_transform(df[columna].fillna('desconocido')).astype(int).toarray()
    else:
        col_encoded = dic[columna].transform(df[columna].fillna('desconocido')).astype(int).toarray()
    col_encoded = pd.DataFrame(col_encoded).add_prefix(columna+ '_enconded_').fillna(0)
    del(df[columna])
    df_nuevo = pd.concat([df.reset_index(drop=True),col_encoded], axis=1)
    return df_nuevo

In [25]:
def one_hot_encode(df, columna, dict_encoding):
    if columna not in dict_encoding:
        dict_encoding[columna] = OneHotEncoder(handle_unknown='ignore')
        dict_encoding[columna].fit_transform(df[[columna]].astype(str)).astype(int)
    col_encoded = (dict_encoding[columna].transform(df[[columna]].astype(str)).astype(int))
    col_encoded = pd.DataFrame(col_encoded.todense()).add_prefix(columna + '_').fillna(0)
    del(df[columna])
    df_nuevo = pd.concat([df.reset_index(drop=True), col_encoded], axis=1)
    return df_nuevo

In [26]:
def mean_encoding(df,columna,label, dict_encoding):
    if columna not in dict_encoding:
        dict_encoding[columna] = TargetEncoder(handle_unknown='ignore')
    df[columna] = dict_encoding[columna].fit_transform(df[[columna]], label)
    df[columna].fillna(0,inplace=True)
    return df

In [27]:
def mean_encoding_val_test(df,columna, dict_encoding):
    df[columna] = dict_encoding[columna].transform(df[[columna]])
    df[columna].fillna(0,inplace=True)
    return df

In [28]:
def label_ec(df,dict_encoding):
    for column in df.columns:
        if df[column].dtypes==object:
            dict_encoding[column] = OrdinalEncoder(handle_unknown='ignore')
            df[column] = dict_encoding[column].fit_transform(df[column].astype(str).values.reshape(-1, 1))
    if df[column].dtype==object:
        df[column] = dict_encoding[column].transform(df[column].astype(str).values.reshape(-1, 1))
    df.fillna(-1,inplace=True)
    return(df)

Establezco listas para cada encoding, en las que almaceno los features que encondeare de cada forma.

In [29]:
countV = ['id_30','id_31','DeviceInfo']

In [30]:
oneHot = ['P_emaildomain','card4','ProductCD','id_38','id_35','id_34']

In [31]:
mean = ['addr2','DeviceType']

#### Creo el modelo

In [32]:
red = MLPClassifier(max_iter=20,random_state=0)

### Train

Realizo los encodings.

In [33]:
for i in countV:
    train=countVect(train,i,dict_encoding)

In [34]:
for i in oneHot:
    train = one_hot_encode(train,i,dict_encoding)

In [35]:
for i in mean:
    train = mean_encoding(train,i,label_train,dict_encoding)

In [36]:
train = label_ec(train,dict_encoding)

Verifico la cantidad de features que tiene mi df para verificar que cumplo con la condicion que deben ser mas de 80.

In [37]:
train.shape

(472432, 532)

Normalizo los features.

In [38]:
train  = scaler.fit_transform(train)

Entreno el modelo.

In [39]:
red.fit(train, label_train)

MLPClassifier(max_iter=20, random_state=0)

Verifico que los 4 features obligatorios ( id_31, id_33, DeviceType, DeviceInfo) esten siendo encodeados.

In [40]:
dict_encoding['id_31']

CountVectorizer(max_features=10)

In [41]:
dict_encoding['id_33']

OrdinalEncoder(handle_unknown='ignore')

In [42]:
dict_encoding['DeviceType']

TargetEncoder(cols=['DeviceType'], handle_unknown='ignore')

In [43]:
dict_encoding['DeviceInfo']

CountVectorizer(max_features=10)

### Validation

Realizo los encodings.

In [44]:
#del(validation['TransactionAmt'])

In [45]:
#del(validation['TransactionDT'])

In [46]:
for i in countV:
    validation=countVect(validation,i,dict_encoding)

In [47]:
for i in oneHot:
    validation = one_hot_encode(validation,i,dict_encoding)

In [48]:
for i in mean:
    validation = mean_encoding_val_test(validation,i,dict_encoding)

In [49]:
validation = label_ec(validation,dict_encoding)

Verifico la cantidad de features que tiene mi df para verificar que cumplo con la condicion que deben ser mas de 80.

In [50]:
validation.shape

(118108, 532)

Normalizo los features.

In [51]:
validation  = scaler.transform(validation)

Muestro el classification_report

In [52]:
print(classification_report(label_validation, red.predict(validation)))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98    114044
           1       0.49      0.36      0.42      4064

    accuracy                           0.97    118108
   macro avg       0.73      0.67      0.70    118108
weighted avg       0.96      0.97      0.96    118108



##### RocAuc 

In [53]:
roc_auc_score(label_validation, red.predict_proba(validation)[:,1])

0.8241424955870741

### Hiperparametros

Realizo la busqueda de hiperparametros.

In [54]:
# from sklearn.model_selection import RandomizedSearchCV
# from scipy.stats import uniform
# mlp = MLPClassifier(max_iter=20,activation='tanh',random_state=0)
# distributions = dict(alpha = [0.0001,0.0002,0.0003,0.0004,0.0005,0.0006],
#                     hidden_layer_sizes = [(80,80,80),(100,10,10),(100,40),(100,)],
#                     learning_rate_init=[0.0005,0.0008,0.001,0.0015,0.002],
#                     momentum = [0.8,0.9,0.95,1],
#                     )
                     
# rs = RandomizedSearchCV(mlp, distributions, scoring='roc_auc',n_jobs=1, verbose=100, cv=3,n_iter=4, random_state=0)
# search = rs.fit(train, label_train)
# search.best_params_

Guardo el modelo de hiperparametros para no volver a correr el RandomizedSearchCV.

In [55]:
# import pickle
# with open('probando_model_2', 'wb') as model_backup_file:
#     pickle.dump(search, model_backup_file)

Cargo el modelo.

In [56]:
import pickle
with open('probando_model_2', 'rb') as file:
    search = pickle.load(file)

Muestro los parametros del modelo encontrado con la busqueda de hiperparametros.

In [57]:
search.best_params_

{'momentum': 0.9,
 'learning_rate_init': 0.002,
 'hidden_layer_sizes': (100, 10, 10),
 'alpha': 0.0002}

***Validation***

Muestro el classification_report con el modelo de hiperparametros.

In [58]:
print(classification_report(label_validation, search.predict(validation)))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98    114044
           1       0.51      0.31      0.39      4064

    accuracy                           0.97    118108
   macro avg       0.74      0.65      0.69    118108
weighted avg       0.96      0.97      0.96    118108



Calculo el ***roc auc*** para el modelo con hiperparametros.

In [59]:
roc_auc_score(label_validation, search.predict_proba(validation)[:,1])

0.8355668315320073